## Setup

In [ ]:
import os
import sys
from functools import partial
import json
from typing import List, Tuple, Union, Optional, Callable, Dict
import torch as t
from torch import Tensor
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import einops
from tqdm import tqdm
from jaxtyping import Float, Int, Bool
from pathlib import Path
import pandas as pd
import circuitsvis as cv
import webbrowser
from IPython.display import display
from transformer_lens import utils, ActivationCache, HookedTransformer, HookedTransformerConfig
from transformer_lens.hook_points import HookPoint
from transformer_lens.components import LayerNorm

from model import create_model
from training import train, TrainArgs
from dataset import MedianDataset
from plotly_utils import hist, bar, imshow

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

## Dataset

In [ ]:
dataset = MedianDataset(size=5, max_num=100, length=7, seed=23)

print("Raw List",dataset[0][:-1])
print("Sorted List",dataset[0][:-1].sort().values)
print("Sorted List Median",dataset[0][:-1].sort().values[dataset.length//2])
print("Label",dataset[0][-1])

## Transformer

In [ ]:
import time

print("Starting training")
start = time.time()
args = TrainArgs(
    max_num=50,
    seq_len=7,
    trainset_size=int(8_000*6),
    valset_size=int(8_000*3),
    epochs=200,
    batch_size=1024,
    lr=3e-3,
    seed=23,
    d_model=32,
    d_head=32,
    n_layers=1,
    n_heads=1,
    d_mlp=None,
    normalization_type="LN",
    use_wandb=True,
    device=device,
)
model = train(args)

print(f"Training took {time.time() - start:.2f} seconds")

In [ ]:
example = t.tensor([45,39,5,]).to(device)
print("Example", example)
print("Label", example.median())
logits = model(example.unsqueeze(0))
out = t.argmax(logits, dim=-1)
print("Prediction", out)

In [ ]:
# Save the model
filename =  "median_model4.pt"
t.save(model.state_dict(), filename)
